In [42]:
import os
import pandas as pd
import git
import numpy as np



# folder = "C:\\Users\\Van\\Desktop\\vancanwin\\pgap3cdg\\01.20.23\\"

def csv_file_adjust(unzipped_folder,filename):
    repo = git.Repo('.', search_parent_directories=True)
    root_folder = repo.working_tree_dir +"\\yeast-data\\"
    folder = root_folder+unzipped_folder+"\\"
    filetype = filename.split(".")[-1]
    if "csv" in filetype:
        df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep="[_,\t ;|:]")#,sep ="[_\t ;|:]",engine='python')
        df.to_csv(folder+"\\"+filename.split(".")[0]+".csv")


def isnumber(x):
    try:
        int(x)
        return True
    except:
        return False
    
def normalize_plate384(unzipped_folder,filename):
    '''Enter file to read plate and normalize data'''
    repo = git.Repo('.', search_parent_directories=True)
    root_folder = repo.working_tree_dir +"\\yeast-data\\"
    folder = unzipped_folder+"\\"
    foldername_sorted = os.path.basename(os.path.normpath(unzipped_folder))
    print("33 foldername", foldername_sorted)

    filetype = filename.split(".")[-1]
    df = pd.DataFrame([])
    
    df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep="[_,\t ;|:]")#,sep ="[_\t ;|:]",engine='python')
    col_num = 30
    index_start = 0
    index_end = 16
    column_index_end = 24
    df_results_index = 0
    for column in range(col_num):
        print("44 DF column number", column, type(column), col_num)
        # print(df.iloc[:,column])
        try:
            df_results_index = df[df.iloc[:,column].str.contains('A', na=False)].index#].isin(['A'])].index#
            column_index = column+1
            print("line 50 matching?",df_results_index)
            break
        except Exception as e:
            print(e)
            
            pass
        # print('checking index', df.iloc[:,column])
    for i, index in enumerate(df_results_index):
        new_header = df.iloc[index-1,column_index:column_index+column_index_end]
        df_results = df.iloc[index+index_start:index+index_end,column_index:column_index+column_index_end] #row,col
        df_results = df_results[df_results.applymap(isnumber)]
        count_na = df_results.isna().sum().sum()
        count_values = df_results.count().sum()

        print("65 DF NA count:", count_na, "65 DF Values count:", count_values, "done")
        try:
            if  count_na < 100 and count_values > 100:
                print("68 DF to normalize",df_results)
                df_results = df_results.fillna(0)
                df_results = df_results.astype(int)
                df_results.columns = new_header
                df_median_neg_controls = int(df_results.iloc[:,0:2].stack().median())
                df_min = df_results.stack().min()
                df_max = df_results.stack().max()
                df_range = df_max-df_min
                df_results_norm = (df_results - df_min)/df_range
                df.iloc[index+index_start:index+index_end,column_index:column_index+column_index_end] = df_results_norm
                print("78 normalized processed...",i)
            else:
                print("80 skip normalized data")
                pass
        except Exception as e: 
            print(e)
            pass

    print("86 normalized data directory and name", unzipped_folder, "and", filename)
    new_filename = unzipped_folder+"\\"+filename#.split(".")[:-1]
    foldername_archive = unzipped_folder.split("\\")[-1]
    print("88 final file directory", new_filename,filename,foldername_sorted)
    filename_directory = root_folder+"!completed\\"+foldername_sorted+"\\"+filename.split(".")[0]+"_normalized."+filename.split(".")[-1]
    print("91 final name", filename_directory)
    df.to_csv(filename_directory)
    
    return(df,filename_directory)

def zscore_plate384(unzipped_folder,file_directory):
    '''Enter file to read plate and normalize data'''
    repo = git.Repo('.', search_parent_directories=True)
    root_folder = repo.working_tree_dir +"\\yeast-data\\"
    folder = unzipped_folder+"\\"
    filetype = "csv"
    filename = file_directory.split("\\")[-1]
    foldername_sorted = os.path.basename(os.path.normpath(unzipped_folder))

    print("100 Filename z-score",folder+filename, filetype,foldername_sorted)

    df = pd.DataFrame([])
    df = pd.read_csv(file_directory, encoding = "utf-8",sep=',')##root_folder+"!completed\\"+filename+"_normalized."+filetype,
    col_num = 30#df.shape[1]
    index_start = 0
    index_end = 16
    column_index_end = 24
  
    for column in range(col_num):
        print("110 z-score column number", column, type(column))
        try:
            df_results_index = df[df.iloc[:,column].str.contains('A', na=False,regex=False)].index
            column_index = column+1
            print("line 114 matching?",df_results_index)
            break
        except Exception as e:
            print(e)
            pass
    for i, index in enumerate(df_results_index):
        new_header = df.iloc[index-1,column_index:column_index+column_index_end]
        df_results = df.iloc[index+index_start:index+index_end,column_index:column_index+column_index_end] #row,col
        count_na = df_results.isna().sum().sum()
        count_values = df_results.count().sum()

        print("125 zscore NA count:", count_na, "Values count:", count_values, "done")
        try:
            if  count_na < 100 and count_values > 100:

                df_results = df_results.fillna(0)
                df_results = df_results.astype(float)

                df_results.columns = new_header
                df_min = df_results.stack().min()
                df_max = df_results.stack().max()
                df_range = df_max-df_min
                df_results_norm = (df_results - df_min)/df_range

                df_mean = df_results_norm.stack().mean()
                df_std = df_results_norm.stack().std()
                df_results_zscore = (df_results_norm - df_mean)/df_std
                df.iloc[index+index_start:index+index_end,column_index:column_index+column_index_end] = df_results_zscore
                print("142 zscore processed...",i)
            else:
                print("144 zscore skip data")
                pass
        except Exception as e: 
            print(e)
            pass


    new_filename = unzipped_folder+"//"+filename#.split(".")[:-1]
    foldername_archive = unzipped_folder.split("\\")[-1]
    print("159 z-score final file directory", new_filename, "\nfoldername:",foldername_archive)
    filename_directory = root_folder+"!completed\\"+foldername_sorted+"\\"+filename.split(".")[0]+"_zscore."+filename.split(".")[-1]
    print("161 z-score final name", filename_directory)
    df.to_csv(filename_directory)
    return(df,filename_directory)


def index_csv(unzipped_folder,file_directory):
    repo = git.Repo('.', search_parent_directories=True)
    root_folder = repo.working_tree_dir +"\\yeast-data\\"    
    folder = unzipped_folder+"\\"
    filetype = "csv"
    filename = file_directory.split("\\")[-1]
    foldername_sorted = os.path.basename(os.path.normpath(unzipped_folder))
    print("161 Filename z-score",folder+filename, filetype)

    df = pd.DataFrame([])
    df = pd.read_csv(file_directory, encoding = "utf-8",sep=',')
    col_num = 30#df.shape[1]
    index_start = 0
    index_end = 16
    column_index_end = 24
    value_list = []
    for column in range(col_num):
        print("171 z-score column number", column, type(column))
        try:
            df_results_index = df[df.iloc[:,column].str.contains('A', na=False,regex=False)].index
            column_index = column+1
            print("line 175 matching?",df_results_index)
            break
        except Exception as e:
            print(e)
            pass
    for i, index in enumerate(df_results_index):
        new_header = df.iloc[index-1,column_index:column_index+column_index_end]
        df_results = df.iloc[index+index_start:index+index_end,column_index:column_index+column_index_end] #row,col
        count_na = df_results.isna().sum().sum()
        count_values = df_results.count().sum()

        print("186 zscore NA count:", count_na, "Values count:", count_values, "done")
        try:
            if  count_na < 100 and count_values > 100:
                print("189 Dimensions: ",df_results.shape[0], df_results.shape[1])
                for row in range(df_results.shape[0]):
                    row_l = chr(ord('@')+row+1)
                    for col in range(df_results.shape[1]):
                        col_l = col+1
                        # print("All results: ", str(row_l)+str(col_l), df_results.iloc[row,col])
                        value_set = [str(row_l)+str(col_l), df_results.iloc[row,col]]
                        value_list.append(value_set)
        except Exception as e: 
            print(e)
            pass
    df_list = pd.DataFrame([value_list]).transpose()
    df_list_2 = pd.DataFrame(df_list.iloc[:,0].to_list(), columns=['index','z-score'])
 
    new_filename = unzipped_folder+"\\!completed\\"+foldername_sorted#.split(".")[:-1]
    foldername_archive = unzipped_folder.split("\\")[-1]
    print("216 list final file directory", new_filename, "\nfoldername:",foldername_sorted)
    filename_directory = root_folder+"!completed\\"+foldername_sorted+"\\"+filename.split(".")[0]+"_list."+filename.split(".")[-1]
    print("217 list final name", filename_directory)
    
    df_list_2.to_csv(filename_directory)#root_folder+"!completed\\"+filename.split(".")[0]+"_list.csv")
    return(df_list_2,filename_directory)

def unzipped_folder_access_files(plate_list, folder_archive):
    repo = git.Repo('.', search_parent_directories=True)
    root_folder = repo.working_tree_dir +"\\yeast-data\\"

    print("length of platelist", len(plate_list))
    n = 0
    for file in plate_list:
        filetype = file.split(".")[-1]
        first_name = file.split(".")[0]
        print("228 extracted files", first_name, filetype)
        if filetype == "zip":
            filename = root_folder+file
            newfilename = "".join(c for c in first_name if c not in '- ')#file.strip(" ").strip('-')
            new_filedirectory = root_folder+newfilename
            print("235",filename, newfilename)
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                print("236 unzipped filename", zip_ref.filename)
                zip_ref.extractall(new_filedirectory)
                print("237", zip_ref, n,new_filedirectory)
                for folder in os.listdir(new_filedirectory):
                    os.replace(new_filedirectory+"\\"+folder,new_filedirectory+"\\"+newfilename)
                    print("242", folder, new_filedirectory)
                shutil.copy(root_folder+file, folder_archive)
                zip_ref.close()
                os.remove(root_folder+file)
                n+=1
                
def new_folder_processed_data(unzipped_folder):
    repo = git.Repo('.', search_parent_directories=True)
    root_folder = repo.working_tree_dir +"\\yeast-data\\"
    foldername_sorted = os.path.basename(os.path.normpath(unzipped_folder))
    folder_unzipped_in_complete = root_folder+"!completed\\"+foldername_sorted+"\\"
    folder_unzipped_in_archive = root_folder+"!archive\\"+foldername_sorted+"\\"
    
    print("244 folder making", folder_unzipped_in_complete)

    try:
        os.mkdir(folder_unzipped_in_complete)
        os.mkdir(folder_unzipped_in_archive)
        os.chdir(root_folder)

    except:
        os.chdir(root_folder)
   

In [ ]:
import os, git, zipfile, shutil
import pandas as pd
import numpy as np

                
repo = git.Repo('.', search_parent_directories=True)
root_folder = repo.working_tree_dir +"\\yeast-data\\"
archive_folder = root_folder+"!archive\\"

plate_list = os.listdir(root_folder)
print("11 list", plate_list)
unzipped_folder_access_files(plate_list, archive_folder)


unzipped_folders_list = os.listdir(root_folder)
print('16', unzipped_folders_list)
for unzipped_foldername in unzipped_folders_list:
    if not any(extension in unzipped_foldername for extension in ("!", "zip")):
        foldername_extracted = unzipped_foldername.split("-")[0]
        print("19 unzipped folder",unzipped_foldername,foldername_extracted)
        unzipped_folder_directory = root_folder+unzipped_foldername+"\\"+unzipped_foldername+"\\"
        print("20 unzipped folder final", unzipped_folder_directory)
        # unzipped_folders = os.listdir(unzipped_folder_directory)
        # print("24 unzipped folder", unzipped_folder)
        new_folder_processed_data(unzipped_folder_directory)
        unzipped_folder = os.listdir(unzipped_folder_directory)
        print("26 unzipped folder list", unzipped_folder)

        for unzipped_file in unzipped_folder:
            filetype = unzipped_file.split(".")[-1]
            print("27 unzipped files", unzipped_file,filetype)
            if filetype == "csv":
                print("33 unzipped files", unzipped_file)
                normalized_directory = normalize_plate384(unzipped_folder_directory,unzipped_file)
                print("35 normalized directory", normalized_directory[1])
                z_score_directory = zscore_plate384(unzipped_folder_directory,normalized_directory[1])
                index_csv(unzipped_folder_directory,z_score_directory[1])
            else:
                print("40 end", unzipped_foldername, unzipped_file)
                pass


11 list ['!archive', '!completed', 'PIGWplate1&19repeat090720230907T203221Z001', 'PIGWRepeat83020230830T215756Z001', 'PIGWRepeat83020230831T194039Z001']
length of platelist 5
228 extracted files !archive !archive
228 extracted files !completed !completed
228 extracted files PIGWplate1&19repeat090720230907T203221Z001 PIGWplate1&19repeat090720230907T203221Z001
228 extracted files PIGWRepeat83020230830T215756Z001 PIGWRepeat83020230830T215756Z001
228 extracted files PIGWRepeat83020230831T194039Z001 PIGWRepeat83020230831T194039Z001
16 ['!archive', '!completed', 'PIGWplate1&19repeat090720230907T203221Z001', 'PIGWRepeat83020230830T215756Z001', 'PIGWRepeat83020230831T194039Z001']
19 unzipped folder PIGWplate1&19repeat090720230907T203221Z001 PIGWplate1&19repeat090720230907T203221Z001
20 unzipped folder final C:\Users\vandu\Desktop\vancanwin\yeast-data\PIGWplate1&19repeat090720230907T203221Z001\PIGWplate1&19repeat090720230907T203221Z001\
244 folder making C:\Users\vandu\Desktop\vancanwin\yeast-d

C:\Users\vandu\AppData\Local\Temp\ipykernel_10920\2176954152.py:38: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep="[_,\t ;|:]")#,sep ="[_\t ;|:]",engine='python')


100 Filename z-score C:\Users\vandu\Desktop\vancanwin\yeast-data\PIGWplate1&19repeat090720230907T203221Z001\PIGWplate1&19repeat090720230907T203221Z001\\19R_normalized.csv csv PIGWplate1&19repeat090720230907T203221Z001
110 z-score column number 0 <class 'int'>
line 114 matching? Int64Index([22, 23, 24, 25, 58, 77, 87, 101, 105, 117, 118], dtype='int64')
125 zscore NA count: 331 Values count: 53 done
144 zscore skip data
125 zscore NA count: 329 Values count: 55 done
144 zscore skip data
125 zscore NA count: 333 Values count: 51 done
144 zscore skip data
125 zscore NA count: 334 Values count: 50 done
144 zscore skip data
125 zscore NA count: 108 Values count: 276 done
144 zscore skip data
125 zscore NA count: 324 Values count: 60 done
144 zscore skip data
125 zscore NA count: 333 Values count: 51 done
144 zscore skip data
125 zscore NA count: 338 Values count: 46 done
144 zscore skip data
125 zscore NA count: 309 Values count: 51 done
144 zscore skip data
125 zscore NA count: 51 Values c

C:\Users\vandu\AppData\Local\Temp\ipykernel_10920\2176954152.py:38: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep="[_,\t ;|:]")#,sep ="[_\t ;|:]",engine='python')


35 normalized directory C:\Users\vandu\Desktop\vancanwin\yeast-data\!completed\PIGWplate1&19repeat090720230907T203221Z001\1R_normalized.csv
100 Filename z-score C:\Users\vandu\Desktop\vancanwin\yeast-data\PIGWplate1&19repeat090720230907T203221Z001\PIGWplate1&19repeat090720230907T203221Z001\\1R_normalized.csv csv PIGWplate1&19repeat090720230907T203221Z001
110 z-score column number 0 <class 'int'>
line 114 matching? Int64Index([22, 23, 24, 25, 58, 77, 87, 101, 105, 117, 118], dtype='int64')
125 zscore NA count: 331 Values count: 53 done
144 zscore skip data
125 zscore NA count: 329 Values count: 55 done
144 zscore skip data
125 zscore NA count: 333 Values count: 51 done
144 zscore skip data
125 zscore NA count: 334 Values count: 50 done
144 zscore skip data
125 zscore NA count: 108 Values count: 276 done
144 zscore skip data
125 zscore NA count: 324 Values count: 60 done
144 zscore skip data
125 zscore NA count: 333 Values count: 51 done
144 zscore skip data
125 zscore NA count: 338 Valu

...

C:\Users\vandu\AppData\Local\Temp\ipykernel_10920\2176954152.py:38: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep="[_,\t ;|:]")#,sep ="[_\t ;|:]",engine='python')


35 normalized directory C:\Users\vandu\Desktop\vancanwin\yeast-data\!completed\PIGWRepeat83020230830T215756Z001\15_normalized.csv
100 Filename z-score C:\Users\vandu\Desktop\vancanwin\yeast-data\PIGWRepeat83020230830T215756Z001\PIGWRepeat83020230830T215756Z001\\15_normalized.csv csv PIGWRepeat83020230830T215756Z001
110 z-score column number 0 <class 'int'>
line 114 matching? Int64Index([22, 23, 24, 25, 58, 77, 87, 101, 105, 117, 118], dtype='int64')
125 zscore NA count: 331 Values count: 53 done
144 zscore skip data
125 zscore NA count: 329 Values count: 55 done
144 zscore skip data
125 zscore NA count: 333 Values count: 51 done
144 zscore skip data
125 zscore NA count: 334 Values count: 50 done
144 zscore skip data
125 zscore NA count: 108 Values count: 276 done
144 zscore skip data
125 zscore NA count: 324 Values count: 60 done
144 zscore skip data
125 zscore NA count: 333 Values count: 51 done
144 zscore skip data
125 zscore NA count: 338 Values count: 46 done
144 zscore skip data
1

C:\Users\vandu\AppData\Local\Temp\ipykernel_10920\2176954152.py:38: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep="[_,\t ;|:]")#,sep ="[_\t ;|:]",engine='python')


35 normalized directory C:\Users\vandu\Desktop\vancanwin\yeast-data\!completed\PIGWRepeat83020230830T215756Z001\2_normalized.csv
100 Filename z-score C:\Users\vandu\Desktop\vancanwin\yeast-data\PIGWRepeat83020230830T215756Z001\PIGWRepeat83020230830T215756Z001\\2_normalized.csv csv PIGWRepeat83020230830T215756Z001
110 z-score column number 0 <class 'int'>
line 114 matching? Int64Index([22, 23, 24, 25, 58, 77, 87, 101, 105, 117, 118], dtype='int64')
125 zscore NA count: 331 Values count: 53 done
144 zscore skip data
125 zscore NA count: 329 Values count: 55 done
144 zscore skip data
125 zscore NA count: 333 Values count: 51 done
144 zscore skip data
125 zscore NA count: 334 Values count: 50 done
144 zscore skip data
125 zscore NA count: 108 Values count: 276 done
144 zscore skip data
125 zscore NA count: 324 Values count: 60 done
144 zscore skip data
125 zscore NA count: 333 Values count: 51 done
144 zscore skip data
125 zscore NA count: 338 Values count: 46 done
144 zscore skip data
125

C:\Users\vandu\AppData\Local\Temp\ipykernel_10920\2176954152.py:38: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep="[_,\t ;|:]")#,sep ="[_\t ;|:]",engine='python')
